<a href="https://colab.research.google.com/github/andreiklychev1/study/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22baseline_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyunpack
!pip install patool
!pip install surprise

     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 11.8MB 8.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618242 sha256=a3808f180a79e97fb54ef13555da8255963e928da38048bb916b624eec6792c5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [3]:
from pyunpack import Archive

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# files.download('data.7z')
Archive('/content/drive/MyDrive/data.7z').extractall('data')

In [7]:
#считывание данных 
train_data = pd.read_csv("data/train_data.csv", sep=';')
train_labels = pd.read_csv("data/train_labels.csv", sep=';')
test_data = pd.read_csv("data/test_data.csv", sep=';')

In [ ]:
# train_data.isna().sum()
# train_data.shape

In [9]:
#Заполнение пропусков строкой 'None'
train_data = train_data.fillna('None')
test_data = test_data.fillna('None')

<hr>

# Подготовка данных 

#### Обработка okpd2 и additional_code

In [10]:
train_data['okpd2_or_additional_code'] = train_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)
test_data['okpd2_or_additional_code'] = test_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)

In [11]:
train_labels

,pn_lot_anon,participant_inn_kpp_anon,is_winner,fz
0,pn_lot_1770702,inn_kpp_3661899,1,44fz
1,pn_lot_1058704,inn_kpp_3661899,1,44fz
2,pn_lot_4186044,inn_kpp_3661899,1,44fz
3,pn_lot_6882961,inn_kpp_3661899,1,44fz
4,pn_lot_5821609,inn_kpp_3661899,1,44fz
...,...,...,...,...
928443,pn_lot_4113734,inn_kpp_9228706,0,44fz
928444,pn_lot_1672363,inn_kpp_9228706,1,223fz
928445,pn_lot_5319307,inn_kpp_9228706,1,223fz
928446,pn_lot_7591446,inn_kpp_2431264,1,223fz


In [13]:
re.search('\d+', train_labels['participant_inn_kpp_anon'].values[0]).group(0)

NameError: ignored

In [18]:
codes = pd.read_excel('коды.xlsx')
codes

,Список регионов,Код региона
0,Республика Адыгея,1
1,Республика Башкортостан,"02, 102"
2,Республика Бурятия,3
3,Республика Алтай,4
4,Республика Дагестан,5
...,...,...
81,Чукотский автономный округ,87
82,Ямало-Ненецкий автономный округ,89
83,Севастополь,92
84,Байконур,94


In [16]:
import re
inns = []
lens = []

for value in train_labels['participant_inn_kpp_anon'].values:
  try:
    inns.append(int(re.search('\d{2}', value).group(0)))
    lens.append(len(re.search('\d+', value).group(0)))
  except:
    pass

inns_lens = pd.DataFrame(data={'inns': inns, 'lens': lens})

In [17]:
codes['Код региона'].values
codes_ = []
regions_ = []
for _ in range(len(codes['Код региона'].values)):
  if type(codes['Код региона'].values[_]) == str:
    for el in re.findall('\d+', codes['Код региона'].values[_]):
      codes_.append(int(el))
      regions_.append(codes['Список регионов'].values[_])

  else:
    codes_.append(codes['Код региона'].values[_])
    regions_.append(codes['Список регионов'].values[_])


NameError: ignored

In [ ]:
codes = pd.DataFrame(data={'Список регионов': regions_, 'Код региона': codes_})
codes

,Список регионов,Код региона
0,Республика Адыгея,1
1,Республика Башкортостан,2
2,Республика Башкортостан,102
3,Республика Бурятия,3
4,Республика Алтай,4
...,...,...
127,Чукотский автономный округ,87
128,Ямало-Ненецкий автономный округ,89
129,Севастополь,92
130,Байконур,94


In [ ]:
# Проверяем наличие регионов
set(inns_lens.inns) - set(codes['Код региона'])

{20}

In [ ]:
#Просмотр частоты значений 
train_data['okpd2_or_additional_code'].value_counts()

drug    65646
32.5    36292
71.1    17836
20.5    15591
26.2    14783
        ...  
07.2        1
52          1
78.0        1
70.0        1
88.1        1
Name: okpd2_or_additional_code, Length: 657, dtype: int64

#### Формирование описания тендера

In [ ]:
train_data['text_description_tender'] = train_data['purchase_name'] + " " + train_data['lot_name'] + " " + train_data['okpd2_names'] + " " + train_data['additional_code_names'] + " " + train_data['item_descriptions']

In [ ]:
train_data.head(3)

,pn_lot_anon,fz,region_code,min_publish_date,purchase_name,lot_name,lot_price,okpd2_code,okpd2_names,additional_code,additional_code_names,item_descriptions,okpd2_or_additional_code,text_description_tender
0,pn_lot_7031618,44fz,2,2019-08-26,Услуги по проведению финансового аудита,None,123500.0,69.2,Услуги по проведению финансового аудита,None,None,Услуги по проведению финансового аудита,69.2,Услуги по проведению финансового аудита None У...
1,pn_lot_7808247,44fz,2,2019-03-12,Приобретение призов для проведения конкурса по...,None,144000.0,26.4,"Телефоны головные, наушники и комбинированные ...",None,None,"Телефоны головные, наушники и комбинированные ...",26.4,Приобретение призов для проведения конкурса по...
2,pn_lot_7009496,44fz,2,2019-03-14,приобретение призов для проведения мероприятия...,None,124200.0,26.2,Устройства запоминающие внешние,None,None,Устройства запоминающие внешние,26.2,приобретение призов для проведения мероприятия...


#### Работа с историей участий поставщика

In [ ]:
inn_kpp_history = pd.merge(train_labels, train_data[['pn_lot_anon','region_code', 'okpd2_or_additional_code']], on=['pn_lot_anon'])

In [ ]:
inn_kpp_history.head()

,index,participant_pn_lot_anon,list_participant_inn_kpp_anon,list_is_winner,list_fz,list_region_code,list_okpd2_or_additional_code
0,0,inn_kpp_1000018,"[pn_lot_5330449, pn_lot_648665, pn_lot_81155, ...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[64, 50, 50, 77, 50, 64, 77, 77, 77, 77, 50, 7...","[58.2, 26.2, 26.4, 26.3, 26.2, 58.2, 26.2, 27...."
1,1,inn_kpp_1001268,"[pn_lot_7608031, pn_lot_9976372, pn_lot_671651...","[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[77, 79, 79, 79, 79, 27, 79, 79, 79, 79, 79, 7...","[86.9, 86.1, 86.2, 86.9, 86.9, 86.9, 86.9, 86...."
2,2,inn_kpp_1002518,"[pn_lot_5287076, pn_lot_7474816, pn_lot_121000...","[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[41, 27, 82, 65, 65, 65, 41, 41, 41, 41, 82, 4...","[30.3, 27.1, 27.1, 25.1, 30.9, 29.2, 25.9, 25...."
3,3,inn_kpp_1003261,[pn_lot_2647783],[1],[223fz],[54],[28.2]
4,4,inn_kpp_1003291,"[pn_lot_7794721, pn_lot_4767800, pn_lot_915979...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[50, 12, 50, 50, 51, 69, 11, 12, 11, 12, 12, 1...","[32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 21...."


In [ ]:
# группировка по поставщику 
inn_kpp_history = inn_kpp_history.groupby('participant_inn_kpp_anon').apply(lambda x: [
    list(x['pn_lot_anon']),
    list(x['is_winner']), 
    list(x['fz']), 
    list(x['region_code']), 
    list(x['okpd2_or_additional_code'])]).apply(pd.Series)

In [ ]:
inn_kpp_history = inn_kpp_history.reset_index()
inn_kpp_history.columns = ['participant_pn_lot_anon', 'list_participant_inn_kpp_anon',
                           'list_is_winner', 'list_fz', 'list_region_code',
                           'list_okpd2_or_additional_code']

ValueError: ignored

In [ ]:
train_data.head()

,pn_lot_anon,fz,region_code,min_publish_date,purchase_name,lot_name,lot_price,okpd2_code,okpd2_names,additional_code,additional_code_names,item_descriptions,okpd2_or_additional_code
0,pn_lot_7031618,44fz,2,2019-08-26,Услуги по проведению финансового аудита,None,123500.0,69.2,Услуги по проведению финансового аудита,None,None,Услуги по проведению финансового аудита,69.2
1,pn_lot_7808247,44fz,2,2019-03-12,Приобретение призов для проведения конкурса по...,None,144000.0,26.4,"Телефоны головные, наушники и комбинированные ...",None,None,"Телефоны головные, наушники и комбинированные ...",26.4
2,pn_lot_7009496,44fz,2,2019-03-14,приобретение призов для проведения мероприятия...,None,124200.0,26.2,Устройства запоминающие внешние,None,None,Устройства запоминающие внешние,26.2
3,pn_lot_5938735,44fz,2,2019-03-14,Продление лицензии на 1С -Битрикс Управление с...,None,50000.0,46.5,Услуги по оптовой торговле программным обеспеч...,None,None,Продление лицензии на 1С -Битрикс Управление с...,46.5
4,pn_lot_9327348,44fz,2,2019-06-26,Приобретение компьютерной техники,None,493441.0,26.2,Машины вычислительные электронные цифровые про...,None,None,монитор || системные блоки системный блок,26.2


#### Рекомендательная система

Выбирается 35 случайный процедур из подвыборки с совпадением региона и ОКПД2 кода актуальной процедуры с регионами и ОКПД2 кодами из истории участия поставщика

In [ ]:
inn_kpp_recommendation = []
similarity_score = 1
for inn_kpp in tqdm_notebook(inn_kpp_history.values):
    participant_inn_kpp_anon, list_participant_inn_kpp_anon, list_is_winner, list_fz, list_region_code, list_okpd2_or_additional_code = inn_kpp
    #подвыборка с совпадением региона и ОКПД2 кода актуальной с историей поставщика
    recommendation = test_data[test_data['region_code'].isin(list_region_code) & test_data['okpd2_or_additional_code'].isin(list_okpd2_or_additional_code)]
    if recommendation.shape[0] >= 35:
        #выбор 35 случайных актуальных процедур из подвыборки
        recommendation = recommendation.sample(35)['pn_lot_anon'].values
        for actual_pn_lot in recommendation:
            inn_kpp_recommendation.append([participant_inn_kpp_anon, actual_pn_lot, similarity_score])

#### Пример формирования файла рекомендаций

In [ ]:
recommendation = pd.DataFrame(inn_kpp_recommendation, columns=['inn_kpp', 'actual_recommended_pn_lot', 'similarity_score'])
recommendation.to_csv("team_name.csv", index=False, sep=';')

In [ ]:
recommendation

,inn_kpp,actual_recommended_pn_lot,similarity_score
0,inn_kpp_1000018,pn_lot_8232437,1
1,inn_kpp_1000018,pn_lot_8852817,1
2,inn_kpp_1000018,pn_lot_4991778,1
3,inn_kpp_1000018,pn_lot_2862283,1
4,inn_kpp_1000018,pn_lot_2023583,1
...,...,...,...
452020,inn_kpp_9999743,pn_lot_9276412,1
452021,inn_kpp_9999743,pn_lot_9074220,1
452022,inn_kpp_9999743,pn_lot_1038393,1
452023,inn_kpp_9999743,pn_lot_2211140,1


# Подсчёт метрик

In [ ]:
test_labels = pd.read_csv("ваша тестирующая выборка", sep=';')

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
true = set((test_labels['pn_lot_anon'] + "_" + test_labels['participant_inn_kpp_anon']).values)

NameError: ignored

In [ ]:
pred = set((recommendation['actual_recommended_pn_lot'] + "_" + recommendation['inn_kpp']).values)

In [ ]:
intersection = len(true.intersection(pred))

In [ ]:
print(f"Точность: {intersection / len(pred) * 100}")
print(f"Полнота: {intersection / len(true) * 100}")

Точность: 1.537968032741552
Полнота: 6.389999540420057
